In [1]:
import pandas as pd
import os
import tqdm
import cvxpy as cv
import numpy as np
cwd = os.getcwd()
cwd

'E:\\backtrader\\scripts'

In [2]:
data_path = r"E:\backtrader\CSMAR\日个股回报率文件\日个股回报率文件\daily_return.csv"

In [3]:
df = pd.read_csv(data_path)
df.head()

,id,date,daily_return_divident,daily_return,pre_close_price
0,1,2018-11-01,-0.007333,-0.007333,10.91
1,1,2018-11-02,0.024007,0.024007,10.83
2,1,2018-11-05,-0.015329,-0.015329,11.09
3,1,2018-11-06,-0.007326,-0.007326,10.92
4,1,2018-11-07,-0.002768,-0.002768,10.84


In [4]:
for column in df.columns:
    print(df[column].dtype)

int64
object
float64
float64
float64


In [5]:
df['date'] = pd.to_datetime(df['date'])

In [6]:
time_intervals = df[df['id'] == 1]['date'].count()
print("交易天数：",time_intervals)
print("证券数：",df['id'].nunique())
df[['date', 'id']].describe()

交易天数： 1213
证券数： 5536


,date,id
count,5366789,5.366789e+06
mean,2021-07-07 18:50:44.922839808,3.468228e+05
min,2018-11-01 00:00:00,1.000000e+00
25%,2020-04-22 00:00:00,2.648000e+03
50%,2021-08-11 00:00:00,3.007490e+05
75%,2022-10-12 00:00:00,6.011630e+05
max,2023-10-31 00:00:00,9.009570e+05
std,NaN,2.763127e+05


In [7]:
time_index = df.loc[df['id'] == 1]['date']
time_index

0      2018-11-01
1      2018-11-02
2      2018-11-05
3      2018-11-06
4      2018-11-07
          ...    
1208   2023-10-25
1209   2023-10-26
1210   2023-10-27
1211   2023-10-30
1212   2023-10-31
Name: date, Length: 1213, dtype: datetime64[ns]

In [8]:
df_ts = df.groupby('id')['date']
df_ts_stock = pd.DataFrame(time_index)
df_ts_stock

,date
0,2018-11-01
1,2018-11-02
2,2018-11-05
3,2018-11-06
4,2018-11-07
...,...
1208,2023-10-25
1209,2023-10-26
1210,2023-10-27
1211,2023-10-30


In [9]:
for k, v in tqdm.tqdm(df_ts.groups.items()):
    df_ts_stock = pd.merge(df_ts_stock, df.iloc[v][['date','daily_return']], on='date', how='left', suffixes=("", f"_{k}"))
df_ts_stock

100%|██████████| 5536/5536 [01:59<00:00, 46.48it/s] 


,date,daily_return,daily_return_2,daily_return_4,daily_return_5,daily_return_6,daily_return_7,daily_return_8,daily_return_9,daily_return_10,...,daily_return_900945,daily_return_900946,daily_return_900947,daily_return_900948,daily_return_900951,daily_return_900952,daily_return_900953,daily_return_900955,daily_return_900956,daily_return_900957
0,2018-11-01,-0.007333,0.007842,-0.002574,-0.007326,-0.005917,-0.012414,-0.013605,0.002488,-0.009132,...,-0.003021,0.006803,0.005780,-0.010943,0.012500,-0.008646,0.000000,0.000000,-0.010000,-0.007974
1,2018-11-02,0.024007,0.008190,0.016774,0.018450,0.019841,0.001397,0.004598,0.027295,0.009217,...,0.018182,0.016892,0.020115,0.016170,0.019753,0.008721,0.015326,0.009615,0.017677,0.019293
2,2018-11-05,-0.015329,-0.023558,0.034898,0.018116,-0.001946,0.065551,0.034325,0.036232,0.004566,...,0.002976,0.000000,-0.011268,-0.008375,-0.002421,0.002882,0.005660,0.015873,-0.007444,-0.006309
3,2018-11-06,-0.007326,0.004160,-0.003066,0.003559,-0.001949,-0.079843,-0.019912,0.027972,-0.013636,...,-0.014837,-0.006645,0.002849,-0.003378,-0.029126,0.011494,-0.037523,-0.012500,-0.015000,-0.025397
4,2018-11-07,-0.002768,-0.012013,-0.008610,-0.010638,-0.007813,0.000000,NaN,-0.022676,0.002304,...,-0.003012,-0.003344,-0.011364,-0.001695,0.002500,0.002841,-0.005848,-0.009494,-0.010152,-0.006515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2023-10-25,-0.016114,0.003431,0.001757,0.014184,0.014052,-0.034137,0.017241,0.000000,0.012690,...,0.000000,0.006135,0.005025,0.017126,NaN,0.020619,0.003289,NaN,NaN,0.017143
1209,2023-10-26,0.002890,-0.011111,0.008772,-0.020979,-0.011547,-0.027027,-0.004237,-0.002988,0.017544,...,0.005650,-0.006098,0.000000,-0.008785,NaN,-0.005051,0.013115,NaN,NaN,0.022472
1210,2023-10-27,0.003842,0.005186,-0.035362,0.014286,0.009346,0.027778,0.004255,0.024975,-0.032020,...,0.022472,0.018405,0.020000,0.005908,NaN,0.005076,0.016181,NaN,NaN,0.016484
1211,2023-10-30,0.000000,-0.002580,0.020433,-0.007042,0.000000,0.014553,0.000000,0.010721,0.000000,...,0.000000,0.018072,0.000000,-0.001468,NaN,-0.005051,0.038217,NaN,NaN,0.062162


In [10]:
df_ts_stock.rename(columns={'daily_return': 'daily_return_1'}, inplace=True)
df_ts_stock

,date,daily_return_1,daily_return_2,daily_return_4,daily_return_5,daily_return_6,daily_return_7,daily_return_8,daily_return_9,daily_return_10,...,daily_return_900945,daily_return_900946,daily_return_900947,daily_return_900948,daily_return_900951,daily_return_900952,daily_return_900953,daily_return_900955,daily_return_900956,daily_return_900957
0,2018-11-01,-0.007333,0.007842,-0.002574,-0.007326,-0.005917,-0.012414,-0.013605,0.002488,-0.009132,...,-0.003021,0.006803,0.005780,-0.010943,0.012500,-0.008646,0.000000,0.000000,-0.010000,-0.007974
1,2018-11-02,0.024007,0.008190,0.016774,0.018450,0.019841,0.001397,0.004598,0.027295,0.009217,...,0.018182,0.016892,0.020115,0.016170,0.019753,0.008721,0.015326,0.009615,0.017677,0.019293
2,2018-11-05,-0.015329,-0.023558,0.034898,0.018116,-0.001946,0.065551,0.034325,0.036232,0.004566,...,0.002976,0.000000,-0.011268,-0.008375,-0.002421,0.002882,0.005660,0.015873,-0.007444,-0.006309
3,2018-11-06,-0.007326,0.004160,-0.003066,0.003559,-0.001949,-0.079843,-0.019912,0.027972,-0.013636,...,-0.014837,-0.006645,0.002849,-0.003378,-0.029126,0.011494,-0.037523,-0.012500,-0.015000,-0.025397
4,2018-11-07,-0.002768,-0.012013,-0.008610,-0.010638,-0.007813,0.000000,NaN,-0.022676,0.002304,...,-0.003012,-0.003344,-0.011364,-0.001695,0.002500,0.002841,-0.005848,-0.009494,-0.010152,-0.006515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2023-10-25,-0.016114,0.003431,0.001757,0.014184,0.014052,-0.034137,0.017241,0.000000,0.012690,...,0.000000,0.006135,0.005025,0.017126,NaN,0.020619,0.003289,NaN,NaN,0.017143
1209,2023-10-26,0.002890,-0.011111,0.008772,-0.020979,-0.011547,-0.027027,-0.004237,-0.002988,0.017544,...,0.005650,-0.006098,0.000000,-0.008785,NaN,-0.005051,0.013115,NaN,NaN,0.022472
1210,2023-10-27,0.003842,0.005186,-0.035362,0.014286,0.009346,0.027778,0.004255,0.024975,-0.032020,...,0.022472,0.018405,0.020000,0.005908,NaN,0.005076,0.016181,NaN,NaN,0.016484
1211,2023-10-30,0.000000,-0.002580,0.020433,-0.007042,0.000000,0.014553,0.000000,0.010721,0.000000,...,0.000000,0.018072,0.000000,-0.001468,NaN,-0.005051,0.038217,NaN,NaN,0.062162


In [11]:
df_cov = df_ts_stock.select_dtypes(include='number')

In [12]:
cov_matrix = df_cov.cov().values

In [13]:
cov_matrix

array([[4.27245864e-04, 2.47474807e-04, 8.02417217e-05, ...,
        1.00380402e-04, 1.09847761e-04, 1.10208210e-04],
       [2.47474807e-04, 4.61610746e-04, 4.69833532e-05, ...,
        1.19367126e-04, 8.73437583e-05, 9.78281553e-05],
       [8.02417217e-05, 4.69833532e-05, 1.23689613e-03, ...,
        1.31403739e-04, 1.57506621e-04, 1.18537555e-04],
       ...,
       [1.00380402e-04, 1.19367126e-04, 1.31403739e-04, ...,
        1.39671711e-03, 2.11264389e-04, 1.21923581e-04],
       [1.09847761e-04, 8.73437583e-05, 1.57506621e-04, ...,
        2.11264389e-04, 5.13558723e-04, 2.66952604e-04],
       [1.10208210e-04, 9.78281553e-05, 1.18537555e-04, ...,
        1.21923581e-04, 2.66952604e-04, 4.26892953e-04]])

In [14]:
mean_vector = df_cov.mean().values

In [15]:
rf = 0.035/365

In [36]:
# 已知数据
risk_free_rate = 0.03/365  # 无风险利率
expected_return_of_portfolio = 0.001  # 风险资产组合的期望收益率

# 转换数据为 cvxpy 所需的格式
mu = mean_vector  # 假设这是风险资产的期望收益率向量
cov_matrix = 0.5 * (cov_matrix + cov_matrix.T)

# 定义变量
weights = cv.Variable(len(mu))

# 定义目标函数
objective = cv.Minimize(cv.quad_form(weights, cov_matrix, assume_PSD=True))

# 定义约束条件
constraints = [
    cv.sum(weights) == 1,  # 权重之和为1
    weights.T @ mu == expected_return_of_portfolio,  # 期望收益率为给定值
    weights >= 0  # 权重大于等于0
]

# 创建问题并求解
problem = cv.Problem(objective, constraints)
problem.solve()

# 输出结果
print("最优权重:", weights.value)
print("最小方差:", problem.value)


ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


SolverError: Workspace allocation error!

In [32]:
weights = cv.Variable((len(mu),1))
cv.quad_form(weights, cov_matrix)

Exception: Invalid dimensions for arguments.

False

fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!
fushu!